In [719]:
import pandas as pd

from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, FactorRange, Select, CustomJS
from bokeh.layouts import column, row
from bokeh.transform import dodge
from bokeh.palettes import Category10

In [720]:
df = pd.read_csv('../data/Titanic/Titanic-Dataset.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


1. Data preparation

In [721]:
df.shape

(891, 12)

In [722]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [723]:
median_age = df['Age'].median()
mode_embarked = df['Embarked'].mode()[0]

df['Age'].fillna(median_age, inplace=True)
df['Embarked'].fillna(mode_embarked, inplace=True)
df['Cabin'].fillna('Unknown', inplace=True)

C:\Users\t-bva\AppData\Local\Temp\ipykernel_12960\2511165320.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(median_age, inplace=True)
C:\Users\t-bva\AppData\Local\Temp\ipykernel_12960\2511165320.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [724]:
def categorize_age(age):
    if age < 18:
        return 'Child'
    elif age < 30:
        return 'Young Adult'
    elif age < 60:
        return 'Adult'
    else:
        return 'Senior'

df['AgeGroup'] = df['Age'].apply(categorize_age)

In [725]:
df['SurvivalRate'] = df.groupby('AgeGroup')['Survived'].transform('mean') * 100

### Visualization

In [726]:
output_file('titanic_visualizations.html')

`Age Group Survival`: Create a bar chart showing survival rates across different age groups.

In [727]:
age_groups = df['AgeGroup'].unique()
age_group_data = df.groupby('AgeGroup')['SurvivalRate'].mean().reindex(age_groups).values

age_group_source = ColumnDataSource(data=dict(AgeGroup=age_groups, SurvivalRate=age_group_data))

In [728]:
age_group_source.data

{'AgeGroup': array(['Young Adult', 'Adult', 'Child', 'Senior'], dtype=object),
 'SurvivalRate': array([32.8125    , 41.77631579, 53.98230088, 26.92307692])}

In [729]:
p1 = figure(x_range=age_groups, title="Survival Rates by Age Group",
            x_axis_label='Age Group', y_axis_label='Survival Rate (%)',
            height=400, width=1000, toolbar_location=None)
p1.vbar(x='AgeGroup', top='SurvivalRate', width=0.9, source=age_group_source, color='navy')

p1.add_tools(HoverTool(tooltips=[("Age Group", "@AgeGroup"), ("Survival Rate", "@SurvivalRate{0.2f}%")]))
p1.xgrid.grid_line_color = None
p1.y_range.start = 0
p1.y_range.end = 100

`Class and Gender`: Create a grouped bar chart to compare survival rates across different classes (1st, 2nd, 3rd) and genders (male, female).

In [730]:
class_gender_data

Sex,Pclass,female,male
0,1,96.808511,36.885246
1,2,92.105263,15.740741
2,3,50.000000,13.544669


In [731]:
class_gender_data = df.groupby(['Pclass', 'Sex'])['Survived'].mean().unstack().reset_index()
class_gender_data.loc[:, ['female', 'male']] = class_gender_data.loc[:, ['female', 'male']]*100
class_gender_source = ColumnDataSource(class_gender_data)

In [732]:
p2 = figure(x_range=[f'Class {i}' for i in class_gender_data['Pclass'].unique()],
            title="Survival Rates by Class and Gender",
            x_axis_label='Class', y_axis_label='Survival Rate (%)',
            height=400, width=1000, toolbar_location=None)

p2.vbar(x=dodge('Pclass', -0.65,  range=p2.x_range), top='female', source=class_gender_source,
   width=0.25, color="#e84d60", legend_label="female")

p2.vbar(x=dodge('Pclass',  -0.35,  range=p2.x_range), top='male', source=class_gender_source,
   width=0.25, color="#718dbf", legend_label="male")

p2.xgrid.grid_line_color = None
p2.y_range.start = 0
p2.y_range.end = 100
p2.legend.title = 'Gender'
p2.legend.orientation = "horizontal"
p2.legend.location = "top_right"

`Fare vs. Survival`: Create a scatter plot with Fare on the x-axis and survival status on the y-axis, using different colors to represent different classes.

In [733]:
fare_survival_source = ColumnDataSource(df)

color_map = {1: Category10[3][0], 2: Category10[3][1], 3: Category10[3][2]}
fare_survival_source.data['color'] = [color_map[cls] for cls in fare_survival_source.data['Pclass']]

In [734]:
p3 = figure(title="Fare vs. Survival",
            x_axis_label='Fare', y_axis_label='Survived',
            tools='pan,box_zoom,reset,save')

p3.scatter(x='Fare', y='Survived', color='color', source=fare_survival_source,
           legend_group='Pclass', fill_alpha=0.6, size=8)


p3.add_tools(HoverTool(tooltips=[("Fare", "@Fare"), ("Survived", "@Survived"), ("Class", "@Pclass")]))
p3.y_range.start = -0.1
p3.y_range.end = 1.1
p3.legend.location = "bottom_right"

In [735]:
layout = column(p1, p2, p3)

show(layout)